In [5]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText

# ---- Settings ----
model_path = "PaddlePaddle/PaddleOCR-VL-1.5"
image_path = "/content/stock_gs200.jpg"
task = "ocr" # Options: 'ocr' | 'table' | 'chart' | 'formula' | 'spotting' | 'seal'
# ------------------

# ---- Image Preprocessing For Spotting ----
image = Image.open(image_path).convert("RGB")
orig_w, orig_h = image.size
spotting_upscale_threshold = 1500

if task == "spotting" and orig_w < spotting_upscale_threshold and orig_h < spotting_upscale_threshold:
    process_w, process_h = orig_w * 2, orig_h * 2
    try:
        resample_filter = Image.Resampling.LANCZOS
    except AttributeError:
        resample_filter = Image.LANCZOS
    image = image.resize((process_w, process_h), resample_filter)

# Set max_pixels: use 1605632 for spotting, otherwise use default ~1M pixels
max_pixels = 2048 * 28 * 28 if task == "spotting" else 1280 * 28 * 28
# ---------------------------

# -------- Inference --------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PROMPTS = {
    "ocr": "OCR:"
}

model = AutoModelForImageTextToText.from_pretrained(model_path, torch_dtype=torch.bfloat16).to(DEVICE).eval()
processor = AutoProcessor.from_pretrained(model_path)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": PROMPTS[task]},
        ]
    }
]
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    images_kwargs={"size": {"shortest_edge": processor.image_processor.min_pixels, "longest_edge": max_pixels}},
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=512)
result = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:-1])
print(result)

Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


Loading weights:   0%|          | 0/608 [00:00<?, ?it/s]

Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


Nasdaq & AMEX
Stocks in bold rose or fell 5% or more

USA Track your investments with our continuously
TODAY updated stocks. Visit us on the web at
.com money.usatoday.com

52-week
High Low Stock Last Change
High Low Stock Last Change

-A-
45.71 32.50 Biomet 36.71 -0.42
2.76 1.20 Biomira 1.46 +0.03
9.07 5.13 BioScrip 8.05 +0.34
68.89 ABX Air n 7.52 -0.10
33.25 12.40 ACMoore 13.58 -1.57
31.38 13.51 ADA—20.96 +3.16
27.14 12.88 ADC Tel rs 23.21 +0.13
30.40 16.70 ADECp 27.32 +0.73
16.45 10.47 AFC Ent s 15.40 -0.14
4.37 4.50 ASE Tst 7.76 +0.40
19.25 12.75 ASM Intl 17.65 -0.03
20.92 13.94 ASML Hld 21.24 +0.46
27.38 16.39 ASV Inc s 26.76 +0.14
19.82 10.47 ATI Tech 17.89 +0.68
33.62 20.53 ATMI Inc 29.95 +1.29
39.20 16.76 ATP O&G 38.40 -0.59
4.24 1.99 AVI Bio 3.62 -0.02
4.24 1.99
